In [27]:
import os

In [ ]:
%pwd

In [3]:
os.chdir('../')

In [ ]:
%pwd

In [37]:
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/saGit1990/classification-using-MLflow-DVC.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME'] = 'fafd6a840e364a6ae6d5357b678a3f3f86b2bc39'
#os.environ['MLFOW_TRACKING_INSECURE_TLS'] = "true"

In [38]:
import tensorflow as tf
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [39]:
# entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int
    repo_owner: str #'saGit1990' 
    repo_name: str #'classification-using-MLflow-DVC

In [40]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [41]:
# configuration manager
class ConfigurationManager:
    def __init__(
            self, 
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
        ):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri='https://dagshub.com/saGit1990/classification-using-MLflow-DVC.mlflow',
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE,
            repo_name= 'classification-using-MLflow-DVC',
            repo_owner='saGit1990'
        )
        return eval_config

In [42]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [43]:
# component
import mlflow.tracking._tracking_service
import mlflow.tracking._tracking_service.utils
import dagshub

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme     
        dagshub.init(repo_owner=self.config.repo_owner, repo_name= self.config.repo_name, mlflow=True)

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [44]:
# pipeline
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2025-02-04 23:48:28,599: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-02-04 23:48:28,618: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-04 23:48:28,628: INFO: common: created directory at: artifacts]
Found 102 images belonging to 2 classes.
7/7 [==============================] - 13s 2s/step - loss: 17.1071 - accuracy: 0.4314
[2025-02-04 23:48:42,198: INFO: common: json file saved at: scores.json]
[2025-02-04 23:48:44,078: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/saGit1990/classification-using-MLflow-DVC "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "saGit1990/classification-using-MLflow-DVC"

[2025-02-04 23:48:44,098: INFO: helpers: Initialized MLflow to track repo "saGit1990/classification-using-MLflow-DVC"]


Repository saGit1990/classification-using-MLflow-DVC initialized!

[2025-02-04 23:48:44,108: INFO: helpers: Repository saGit1990/classification-using-MLflow-DVC initialized!]


2025/02/04 23:48:45 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-02-04 23:48:49,378: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\SUEL~1.ABB\AppData\Local\Temp\tmpgv30u13r\model\data\model\assets
[2025-02-04 23:48:51,774: INFO: builder_impl: Assets written to: C:\Users\SUEL~1.ABB\AppData\Local\Temp\tmpgv30u13r\model\data\model\assets]


Registered model 'VGG16Model' already exists. Creating a new version of this model...
2025/02/04 23:54:40 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 2
Created version '2' of model 'VGG16Model'.
